# Market Capitalisation
This notebook utilises web scrapping techniques to derive the market capitalisation for companies listed in our group project. 

In [ ]:
# Installing the necessary dependency
!pip install yfinance
!pip install currency

In [ ]:
import pandas as pd
import requests
import yfinance as yf
from currency_converter import CurrencyConverter

Connect to googledrive so as to import csv files with the ticker symbol for each company

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/My Drive/BT4222/mktcapwithticker.csv'
data = pd.read_csv(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data2=data[["Companies","Ticker"]]
data2

,Companies,Ticker
0,24SevenOffice Group,719.F
1,"2U, Inc",TWOU
2,Absolute Software Corp,ABST
3,Accenture,ACN
4,"ACI Worldwide, Inc",ACIW
...,...,...
474,Zensar Technologies Ltd,ZENSARTECH.BO
475,"Ziff Davis, Inc",ZD
476,Zillow Group Inc,Z
477,Zix Corp,ZIXI


Segment the frames into batch size 100. Rationale for doing so is because Yahoo finance allows a maximum of 100 tickers per api call 

In [ ]:
S = 100 # size that i want
N = int(len(data)/S)
frames = [data2.iloc[i*S:(i+1)*S].copy() for i in range(N+1) ]
print(len(data2))
print(len(frames[0]))

479
100


In [ ]:
print(frames[0])

                 Companies    Ticker
0      24SevenOffice Group     719.F
1                  2U, Inc      TWOU
2   Absolute Software Corp      ABST
3                Accenture       ACN
4       ACI Worldwide, Inc      ACIW
..                     ...       ...
95           Coursera, Inc      COUR
96    Crayon Group Holding  CRAYN.OL
97                  Criteo      CRTO
98       CSAM Health Group   CSAM.OL
99   CWT International Ltd       NaN

[100 rows x 2 columns]


Steps to scrape the market capitalisation data:
- Input the api key
- Change the list rating to which is being used currently
- Change frame[x] to the respective frame 

In [ ]:
c = CurrencyConverter()

def getStock(search_term):
    results = []
    query = requests.get(f'https://yfapi.net/v6/finance/autocomplete?region=IN&lang=en&query={search_term}', 
    headers={
        'accept': 'application/json',
        'X-API-KEY': 'Z7cly4ZddSTNL24yAJuO7K5412pyay71cJ1kqp41'
    })
    response = query.json()
    for i in response['ResultSet']['Result']:
          final = i['symbol']
          results.append(final)
          stock = yf.Ticker(results[0])
    try:
       
        mktcap = stock.info['marketCap']
        currency = stock.info["currency"]
        #yfinance outputs currency for GBP as GBp. We will need to change it for the currency conversion.It also doesnt have TWD
        if currency == "GBp":
          mktcap =c.convert(mktcap,'GBP','USD')
        elif currency =='TWD':
          mktcap = mktcap*0.03460263 
        else:
          mktcap = c.convert(mktcap,currency,'USD')
        # price = stock.info["regularMarketPrice"]
        # full_name = stock.info['longName']
        # currency = stock.info["currency"]
        return mktcap
    except Exception as e:
        # return (f"Company cannot be retrieved{stock}")
        return 0
# rating4=[]
# for row in list(frames[4]["Ticker"]):
#   stock = row
#   final = getStock(stock)
#   rating4.append(final)
stock=input("ticker")
mktcap=getStock(stock)
print(mktcap)

ticker6533.TW
692299852.1237915


In [ ]:
len(rating4)

79

Combine all lists into original

In [ ]:
#function to copy all the ratings together
result = rating0+rating1+rating2+rating3+rating4
result
len(result)

479

Append data to original dataframe

In [ ]:
#add back into original dataframe
data2["mktcapUSD"] = result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Companies,Ticker,mktcapUSD
0,24SevenOffice Group,719.F,9.212487e+07
1,"2U, Inc",TWOU,9.673635e+08
2,Absolute Software Corp,ABST,4.188821e+08
3,Accenture,ACN,2.166511e+11
4,"ACI Worldwide, Inc",ACIW,3.068757e+09
...,...,...,...
474,Zensar Technologies Ltd,ZENSARTECH.BO,1.156896e+09
475,"Ziff Davis, Inc",ZD,4.538474e+09
476,Zillow Group Inc,Z,1.205327e+10
477,Zix Corp,ZIXI,6.522908e+08


function to export to csv

In [ ]:
data2

,Companies,Ticker,mktcapUSD
0,24SevenOffice Group,719.F,9.212487e+07
1,"2U, Inc",TWOU,9.673635e+08
2,Absolute Software Corp,ABST,4.188821e+08
3,Accenture,ACN,2.166511e+11
4,"ACI Worldwide, Inc",ACIW,3.068757e+09
...,...,...,...
474,Zensar Technologies Ltd,ZENSARTECH.BO,1.156896e+09
475,"Ziff Davis, Inc",ZD,4.538474e+09
476,Zillow Group Inc,Z,1.205327e+10
477,Zix Corp,ZIXI,6.522908e+08


In [ ]:
data2.to_csv('mktcap_file5.csv', index=False)